In [55]:
import nltk
import fileinput

In [56]:
def to_after(tokens, reserved='about'):
    def to_after_token(token):
        token = token.replace('\u3000', ' ')
        if token == ' ': return ''
        
        if token.endswith('-]'):
            return None
        # key part
        elif token.endswith('>>' + reserved + '+}') or token == '{+' + reserved + '+}':
            return token
        elif token.endswith('+}'):
            return token[token.rfind('>>')+2:-2]  if token.startswith('[-') else token[2:-2]  
        else:
            return token
        
    # after_tokens
    return [e for token in map(to_after_token, tokens) if token for e in token.split(' ')]

In [59]:
to_after(['hello','[-about-]', '[-asdf>>about+}', '[-vvv>>ddd+}', 'asdfdf'], 'about')

['hello', '[-asdf>>about+}', 'ddd', 'asdfdf']

In [81]:
# 擔心有一句包含兩個 about+} 所以換個邏輯
def divide_triedit_noedit(aft_tokens, reserved='about'):
    noedit_list, triedit_list = [[]], []

    for i, tk in enumerate(aft_tokens):
        if '+}' in tk: # reserved word ，應該是不會有 [--] 出現
            if i > 0 and i+1 < len(aft_tokens):
                triedit_list.append(tuple(aft_tokens[i-1:i+2]))
            
            noedit_list.append([])
        else: 
            noedit_list[-1].append(tk)

    return noedit_list, triedit_list

In [85]:
divide_triedit_noedit(['hello', '[-asdf>>about+}', 'ddd', 'asdfdf', '[-fff>>about+}', 'asdss'])
divide_triedit_noedit(['hello', '[-asdf>>about+}', 'ddd', 'asdfdf', 'asdss'])
divide_triedit_noedit(['hello', '[-asdf>>about+}', 'ddd', 'asdfdf', 'asdss', '[-asdsfas>>about+}'])

([['hello'], ['ddd', 'asdfdf', 'asdss'], []],
 [('hello', '[-asdf>>about+}', 'ddd')])

In [92]:
def get_bigram(tokens):
    return list(nltk.bigrams(tokens))

def get_trigram(tokens):
    return list(nltk.trigrams(tokens))

In [96]:
get_trigram(['one'])
get_trigram(['one', 'asd', 'aaa'])

[('one', 'asd', 'aaa')]

In [ ]:
def bi_vs_tri(bigram, uniq_edit_list, reserved='about'):
    bi_com_tri = set()
    for edit in set(uniq_edit_list):
        if edit[1].endswith(reserved + '+}'):
            if bigram[0] == edit[0] and bigram[1] == edit[2]:
                bi_com_tri.add(bigram)
                bi_com_tri.add(edit)
    return sorted(bi_com_tri, key = len)

def tri_vs_tri(tri_noedit, triedit_list):
    tri_com_tri = set()
    for tri_edit in set(triedit_list):
        tri_befs = []
        for token in tri_edit:
            if token.startswith('[-about'):
                token_bef = tri_edit[1][2:-2].split('>>')[0]
                tri_befs.append(token_bef)
            else:
                tri_befs.append(token)  
            
        if tri_noedit == tuple(tri_befs):
            tri_com_tri.add(tri_edit)
            tri_com_tri.add(tri_noedit)
                      
    return sorted(tri_com_tri, reverse = True)

In [94]:
bi_vs_tri([])

['a', 'cdd']

In [46]:



def counting_bvst(bitri, bi_noedit_list, triedit_list):
    dict = {}
    dict[bitri[0]] = bi_noedit_list.count(bitri[0])
    for ngram in bitri[1:]:
        dict[ngram] = [triedit_list.count(ngram), triedit_list.count(ngram)/dict[bitri[0]]]
    return sorted(dict.items(), key = lambda item: len(item[0]))


def counting_tvst(tritri, tri_noedit_list, triedit_list):
    dict = {}
    dict[tritri[0]] = tri_noedit_list.count(tritri[0]) 
    for ngram in tritri[1:]:
        dict[ngram] = [triedit_list.count(ngram), triedit_list.count(ngram)/dict[tritri[0]]]
    return sorted(dict.items(), key = lambda item: len(item[0]))


def sort(counts_bi):
    return sorted(counts_bi, key = lambda item: (item[1][1][1]))



In [9]:

if __name__ == '__main__':
    edit_list = [] # trigram edit

    bigrams, trigrams = [], []
    
    
    triedit_bef_list = []
    bi_com_tri_set = set()
    tri_com_tri_set = set()

    counts_bi = []
    counts_tri = []

    for line in open('ef.diff.simplize.despace.txt', 'r', encoding='utf8').readlines():# fileinput.input():
        tokens = line.strip().split(' ')
        # print(tokens)
        aft_tokens = to_after(tokens)

        noedit_list, triedit_tokens = divide_triedit_noedit(aft_tokens)
        # print(noedit_list)
        # print(triedit_tokens)
        edit_list.extend(triedit_tokens)
        # print(edit_list)

        # 會有 [] 出現，若是長度不夠
        for no_edit in noedit_list:
            bigrams.extend(get_bigram(no_edit))
            trigrams.extend(get_trigram(no_edit))

    # print(edit_list)  
    # print(bigrams)      
    uniq_edit = set(edit_list)
    for bi in set(bigrams):
    ###################################
        bi_com_tri = bi_vs_tri(bi, uniq_edit)
        
    
        if bi_com_tri:
    #         # print(bi_com_tri) 
            bi_com_tri_set.add(tuple(bi_com_tri))
    #         # print('='*50)        

    for tri_noedit in set(tri_noedit_list):
        tri_com_tri = tri_vs_tri(tri_noedit, triedit_list)
        if tri_com_tri:        
            tri_com_tri_set.add(tuple(tri_com_tri))

    # print(bi_com_tri_set)      
    
    for bitri in bi_com_tri_set:
    #     # print(bitri)
        count_bvst = counting_bvst(bitri, bi_noedit_list, triedit_list)
        counts_bi.append(count_bvst)
        # print(count_bvst)
        # print('='*50)

     # print(tri_noedit_list)
    print(tri_com_tri_set)   
    #  
    for tritri in tri_com_tri_set:
    #     # print(tritri)
        count_tvst = counting_tvst(tritri, tri_noedit_list, triedit_list)
    #     # print(count_tvst)
        # counts_tri.append(count_tvst)

        for freq in count_tvst:
            print(' '.join(freq[0]), ': ', freq[1], sep = '')
        print('='*50)


    print(counts_bi)
    print(sort(counts_bi))
    print(counts_tri)
    # print(sort(counts_tri))


    for count_bi in sort(counts_bi):
        for freq in count_bi:
            print(' '.join(freq[0]), ': ', freq[1], sep = '')
        print('='*50)

    # for count_tri in sort(counts_tri):
    #     for freq in count_tri:
    #         print(' '.join(freq[0]), ': ', freq[1], sep = '')
    #     print('='*50)


        # for freq in count_bvst:
        #     print(' '.join(freq[0]), ': ', freq[1], sep = '')
        # print('='*50)

   
        # for freq in count_tvst:
        #     print(' '.join(freq[0]), ': ', freq[1], sep = '')
        # print('='*50)



KeyboardInterrupt: 